#### NN Analysis - calculations on saved models (run on CPU)

- Saliency
- proximal operator error
- objective function error

#### System Set-Up

In [ ]:
import numpy as np
from matplotlib import pyplot as plt 
import time as time
import sys
import platform
import psutil

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, Subset

from captum.attr import Saliency

from sklearn.model_selection import train_test_split

sys.path.append('..')

from prox_op import prox_op
from data_fcns import generate_raw_data
from data_fcns import vanilla_scaling
from data_fcns import compute_features

In [ ]:
# python version

print(sys.version)

In [ ]:
# get CPU info

print(platform.processor())
print(platform.machine())
print(platform.version())
print(platform.platform())
print(platform.uname())
print(platform.system())
print(str(round(psutil.virtual_memory().total / (1024.0 **3)))+" GB")


In [ ]:
# get GPU info

print(torch.cuda.is_available())

if torch.cuda.is_available():
    print(torch.cuda.device_count())
    print(torch.cuda.current_device())
    print(torch.cuda.device(0))
    print(torch.cuda.get_device_name(0))

#### Data Prep

In [ ]:
nn_type = "feature"  # vanilla or feature
data_dist = "norm"   # norm, unif, or both
unif_min = 0
unif_max = 20
min_len = 1000
max_len = 2000
num_vec = 10000
seed = 1

output_path = "models/features/gaussian/len_1000_2000/"
plot_title = "$N(0,1)$ Vectors, Lengths 1,000 - 2,000"

#output_path = "models/features/gaussian/len_1000_100000/"
#plot_title = "$N(0,1)$ Vectors, Lengths 1,000 - 100,000"

#output_path = "models/features/uniform_0_1/len_1000_2000/"
#plot_title = "$U(0,1)$ Vectors, Lengths 1,000 - 2,000"

#output_path = "models/features/uniform_0_1/len_1000_100000/"
#plot_title = "$U(0,1)$ Vectors, Lengths 1,000 - 100,000"

#output_path = "models/features/both/len_1000_2000/"
#plot_title = "$N(0,1)$ and $U(0,1)$ Vectors, Lengths 1,000 - 2,000"

#output_path = "models/features/both/len_1000_100000/"
#plot_title = "$N(0,1)$ and $U(0,1)$ Vectors, Lengths 1,000 - 100,000"

#output_path = "models/density/unif_0_10/len_1000_2000/"
#plot_title = "$U(0,10)$ Vectors, Lengths 1,000 - 2,000"

#output_path = "models/density/unif_0_10/len_1000_100000/"
#plot_title = "$U(0,10)$ Vectors, Lengths 1,000 - 100,000"

#output_path = "models/density/unif_0_20/len_1000_2000/"
#plot_title = "$U(0,20)$ Vectors, Lengths 1,000 - 2,000"

#output_path = "models/density/unif_0_20/len_1000_100000/"
#plot_title = "$U(0,20)$ Vectors, Lengths 1,000 - 100,000"

X, lengths, alphas, taus = generate_raw_data(data_dist, min_len, max_len, num_vec, unif_min, unif_max, seed)

In [ ]:
if nn_type == "vanilla":

    num_moments = 0 # needed for the NN selection cell
    M, yhat, zero_idx = vanilla_scaling(X, lengths, alphas, taus)
    
else:  # features NN
    num_moments = 10
    M, yhat, mus, zero_idx = compute_features(X, lengths, alphas, taus, num_moments)


In [ ]:
# remove any observations from dataset that have tau = 0 

if sum(zero_idx) > 0:
    M = M[~zero_idx,:]
    yhat = yhat[~zero_idx]
    mus = mus[~zero_idx]
    alphas = alphas[~zero_idx]
    taus = taus[~zero_idx]
    

if data_dist == "norm":
    num_norm_vec = M.shape[0]
elif data_dist == "unif":  
    num_norm_vec = 0
else: # both
    num_norm_vec = int(num_vec/2) - sum(np.where(zero_idx)[0] < int(num_vec/2))
    
print(num_norm_vec)

#### Train/Test Data Prep

In [ ]:
np.random.seed(0)
torch.manual_seed(0)

In [ ]:
class TauDataset(Dataset):
    def __init__(self, X, y, num_norm_vec):
        
        d = np.zeros(len(y))
        d[:num_norm_vec] = 1  # Gaussian - 1, Uniform - 0
          
        self.features = torch.Tensor(X)
        self.labels = torch.Tensor(y) 
        self.dist = torch.Tensor(d)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx,:], self.labels[idx], self.dist[idx], idx

In [ ]:
# create dataset
dataset = TauDataset(M,yhat,num_norm_vec)

print(dataset[0])
print(len(dataset))

In [ ]:
if data_dist == "both":

    # train-test split: 80-20, stratified sampling on distribution type
    # Gaussian: 1st half of indices, Uniform: 2nd half of indices 

    train_idx, test_idx = train_test_split(range(len(dataset)), test_size=0.20, random_state=0, 
                                       shuffle=True, stratify=dataset.dist)
    
else: # unif or norm data
    
    # train-test split: 80-20
    train_idx, test_idx = train_test_split(range(len(dataset)), test_size=0.20, random_state=0, shuffle=True)


In [ ]:
sum(dataset.dist[train_idx])

In [ ]:
sum(dataset.dist[test_idx])

In [ ]:
train_data = Subset(dataset, train_idx)
test_data = Subset(dataset, test_idx)

In [ ]:
# set up dataset iterator

train_batch_size = 32
test_batch_size = len(test_data)

train_loader = DataLoader(dataset=train_data, batch_size=train_batch_size, shuffle=True) 
test_loader = DataLoader(dataset=test_data, batch_size=test_batch_size, shuffle=True)


In [ ]:
print(len(train_loader))
print(len(test_loader))

#### Load NN

In [ ]:
# first layer number of inputs
print(dataset[0][0].size())
layer1_size = M[0].shape[0]
layer1_size

In [ ]:
device = "cpu"

In [ ]:
# set NN based on layer1_size

if layer1_size == (num_moments+3):  # features NN
    
    class NeuralNetwork(nn.Module):
        def __init__(self):
            super().__init__()
            self.linear_relu_stack = nn.Sequential(
                nn.Linear(num_moments+3, 25),  
                nn.ReLU(),    
                nn.Linear(25, 10),
                nn.ReLU(),
                nn.Linear(10, 1)
            )

        def forward(self, x):
            tau = self.linear_relu_stack(x) 
            return tau
        
elif (layer1_size == 2000) or (layer1_size == 100000):   # vanilla NN
    
    class NeuralNetwork(nn.Module):
        def __init__(self):
            super().__init__()
            self.linear_relu_stack = nn.Sequential( 
                nn.Linear(layer1_size, 200),   
                nn.ReLU(),  
                nn.Linear(200, 100),
                nn.ReLU(),
                nn.Linear(100, 50), 
                nn.ReLU(),
                nn.Linear(50, 1) 
            )

        def forward(self, x):
            tau = self.linear_relu_stack(x)
            return tau
    
else:
    pass

model = NeuralNetwork().to(device)
print(model)

In [ ]:
# load model

model = NeuralNetwork()
model.load_state_dict(torch.load("models/features/gaussian/len_1000_2000/epoch_4421_nn.pt"))
#model.load_state_dict(torch.load("models/features/gaussian/len_1000_100000/epoch_4695_nn.pt"))
#model.load_state_dict(torch.load("models/features/uniform_0_1/len_1000_2000/epoch_4897_nn.pt"))
#model.load_state_dict(torch.load("models/features/uniform_0_1/len_1000_100000/epoch_4928_nn.pt"))
#model.load_state_dict(torch.load("models/features/both/len_1000_2000/epoch_4570_nn.pt"))
#model.load_state_dict(torch.load("models/features/both/len_1000_100000/epoch_4792_nn.pt"))

# density models
#model.load_state_dict(torch.load("models/density/unif_0_10/len_1000_2000/epoch_3788_nn.pt"))
#model.load_state_dict(torch.load("models/density/unif_0_10/len_1000_100000/epoch_4390_nn.pt"))
#model.load_state_dict(torch.load("models/density/unif_0_20/len_1000_100000/epoch_3779_nn.pt"))
#model.load_state_dict(torch.load("models/density/unif_0_20/len_1000_2000/epoch_4551_nn.pt"))

model.eval()

#### NN Interpretability - Feature Importance using saliency

In [ ]:
saliency = Saliency(model)

In [ ]:
test_features = dataset.features[test_idx] 
test_features.requires_grad_()

In [ ]:
attribution = saliency.attribute(test_features)
print(attribution.shape)

In [ ]:
attr = attribution.detach().numpy()

print(type(attr))
print(attr.shape)

In [ ]:
# plot code - https://captum.ai/tutorials/Titanic_Basic_Interpret

# Helper method to print importances and visualize distribution
def visualize_importances(feature_names, importances, title="Average Feature Importances", plot=True, xaxis_title="Features", yaxis_title="Saliency"):
    print(title)
    for i in range(len(feature_names)):
        print(feature_names[i], ": ", '%.3f'%(importances[i]))
    x_pos = (np.arange(len(feature_names)))
    if plot:
        plt.figure(figsize=(12,6))
        
        width = 0.5 
        plt.bar(x_pos, importances, label='importance', align='center', zorder=2)
    
        plt.xticks(x_pos, feature_names, fontsize=24, rotation = 60)
        plt.yticks(fontsize=24)
        plt.xlabel(xaxis_title, fontsize = 24)
        plt.ylabel(yaxis_title, fontsize = 24)
        plt.title(title, fontsize=22)     
        plt.grid(zorder=0)
        plt.savefig(output_path + "feat_importance.eps", bbox_inches='tight')

In [ ]:
if nn_type == "vanilla":

    feature_names = []
    for i in range(1, attr.shape[1] + 1):
        feature_names = feature_names + [f'{i}'] 

else: # features NN

    feature_names = ["min", "max", "L1 norm", "2nd m.", "3rd m."]
    for i in range(4, num_moments+1):
        feature_names = feature_names + [f'{i}th m.']
    feature_names = feature_names + ["length"] 
    
#feature_names

In [ ]:
visualize_importances(feature_names, np.mean(attr, axis=0), title=plot_title)

#### Proximal operator and Objective Function Errors 

Filtered to testing set at end 

In [ ]:
inf_data = dataset.features
inf_labels = dataset.labels

In [ ]:
# nn inference to get predicted tau_hat

model.eval()

with torch.no_grad():
    pred_tau_hat = model(inf_data)


In [ ]:
# nn output
pred_tau_hat = pred_tau_hat.squeeze().numpy()
pred_tau_hat

In [ ]:
# transform nn output to original tau: 

if nn_type == "vanilla":
    # tau = alpha(tau_hat)
    pred_tau = np.multiply(alphas, pred_tau_hat)
    
else: # features
    # tau = alpha(tau_hat + mu)
    pred_tau = np.add(pred_tau_hat, mus)    
    pred_tau = np.multiply(alphas, pred_tau)

In [ ]:
pred_tau

In [ ]:
# compute proximal operator with known tau

def prox_op_tau(x, tau):
    
    # compute proximal operator
    prox = x.copy()
    idx = (np.abs(x) > tau)
    prox[idx] = np.sign(x[idx])*tau
            
    return prox

In [ ]:
X = X[~zero_idx,:]
lengths = lengths[~zero_idx]

In [ ]:
# compute prox operators and calculate 1) prox op error, 2) objective function error

m = len(taus)

prox_err = np.zeros(m)
prox_relerr = np.zeros(m)
obj_fcn_err = np.zeros(m)
obj_fcn_relerr = np.zeros(m)

for i in range(m):
    
    raw_vec = X[i,0:lengths[i]]
    
    p = prox_op_tau(raw_vec, taus[i])
    pred_p = prox_op_tau(raw_vec, pred_tau[i])
    prox_err[i] = np.linalg.norm(pred_p - p)  # 2-norm by default
    prox_relerr[i] = np.linalg.norm(pred_p - p)/np.linalg.norm(p) 
    
    obj = (1/2)*np.linalg.norm(raw_vec-p)**2 + alphas[i]*np.linalg.norm(p, np.inf)
    pred_obj = (1/2)*np.linalg.norm(raw_vec-pred_p)**2 + alphas[i]*np.linalg.norm(pred_p, np.inf)
    obj_fcn_err[i] = pred_obj - obj
    obj_fcn_relerr[i] = (pred_obj - obj)/obj
    

In [ ]:
prox_err

In [ ]:
prox_relerr

In [ ]:
obj_fcn_err

In [ ]:
obj_fcn_relerr

In [ ]:
print(np.median(prox_relerr[test_idx]))
print(np.mean(prox_relerr[test_idx]))
print(np.std(prox_relerr[test_idx]))

In [ ]:
print(np.median(obj_fcn_relerr[test_idx]))
print(np.mean(obj_fcn_relerr[test_idx]))
print(np.std(obj_fcn_relerr[test_idx]))